In [22]:
!pip install osmnx geopandas shapely pyproj rtree rasterio matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 135.1 MB/s eta 0:00:0000:01


In [2]:
"""
Energy-relevant geospatial feature extraction pipeline
Region: Karlsruhe KIT North Campus
Author: (Your Name)
"""

# =============================================================================
# 1. IMPORTS & BASIC SETUP
# =============================================================================

import os
import logging
import requests
import numpy as np

import osmnx as ox
import geopandas as gpd
import rasterio
from rasterio.mask import mask

import folium
from shapely.geometry import box

# Logging (safe replacement for deprecated ox.config)
logging.basicConfig(level=logging.INFO)

# =============================================================================
# 2. REGION OF INTEREST (BOUNDING BOX)
# =============================================================================
# KIT North Campus (Eggenstein-Leopoldshafen), conservative bounding box

NORTH = 49.12
SOUTH = 49.07
EAST  = 8.45
WEST  = 8.39

roi_polygon = box(WEST, SOUTH, EAST, NORTH)
roi_gdf = gpd.GeoDataFrame({"geometry": [roi_polygon]}, crs="EPSG:4326")

# =============================================================================
# 3. OSM FEATURE DOWNLOAD (ENERGY-RELEVANT ONLY)
# =============================================================================

tags = {
    "power": True,
    "building": True,
    "amenity": ["charging_station"],
    "man_made": ["pipeline"],
    "landuse": [
        "industrial", "quarry", "landfill", "farmland",
        "forest", "orchard"
    ],
    "natural": ["water", "wood", "scrub"],
    "generator:source": [
        "solar", "wind", "biomass", "biogas",
        "gas", "coal", "oil", "nuclear", "hydro"
    ]
}

logging.info("Downloading OSM features...")
osm_all = ox.features_from_bbox(
    north=NORTH,
    south=SOUTH,
    east=EAST,
    west=WEST,
    tags=tags
)


# Ensure GeoDataFrame
osm_all = osm_all.to_crs("EPSG:4326")

# =============================================================================
# 4. SAFE FEATURE FILTERING (NO KEY ERRORS)
# =============================================================================

def safe_filter(gdf, column, value):
    """Safely filter GeoDataFrame for tag=value"""
    if column in gdf.columns:
        return gdf[gdf[column] == value]
    return gdf.iloc[0:0]

# ---- Buildings with flat roofs
flat_roofs = osm_all[
    (osm_all.get("building").notna()) &
    (osm_all.get("roof:shape") == "flat")
]

# ---- Power infrastructure
power_features = osm_all[osm_all.get("power").notna()]

# ---- Solar generators
solar_generators = osm_all[osm_all.get("generator:source") == "solar"]

# ---- EV charging stations
charging_stations = safe_filter(osm_all, "amenity", "charging_station")

# ---- Pipelines
pipelines = safe_filter(osm_all, "man_made", "pipeline")

# ---- Landuse polygons
landuse_areas = osm_all[osm_all.get("landuse").notna()]

logging.info(f"Flat roofs: {len(flat_roofs)}")
logging.info(f"Power features: {len(power_features)}")
logging.info(f"Solar generators: {len(solar_generators)}")
logging.info(f"EV chargers: {len(charging_stations)}")

# =============================================================================
# 5. VIIRS FIRE RADIATIVE POWER (DIRECT PUBLIC API)
# =============================================================================

frp_gdf = None
try:
    logging.info("Fetching VIIRS Fire Radiative Power (FRP)...")

    viirs_url = (
        "https://services9.arcgis.com/HO7DT5zlZVjMOiAv/arcgis/rest/services/"
        "USA_VIIRS_375/FeatureServer/0/query"
    )

    params = {
        "where": "1=1",
        "outFields": "*",
        "geometry": f"{WEST},{SOUTH},{EAST},{NORTH}",
        "geometryType": "esriGeometryEnvelope",
        "inSR": "4326",
        "outSR": "4326",
        "spatialRel": "esriSpatialRelIntersects",
        "f": "geojson"
    }

    response = requests.get(viirs_url, params=params, timeout=30)
    response.raise_for_status()

    frp_gdf = gpd.GeoDataFrame.from_features(
        response.json()["features"],
        crs="EPSG:4326"
    )

    logging.info(f"VIIRS FRP points: {len(frp_gdf)}")

except Exception as e:
    logging.warning(f"VIIRS FRP unavailable: {e}")

# =============================================================================
# 6. COPERNICUS PLACEHOLDER (STRUCTURED, SAFE)
# =============================================================================
"""
Copernicus datasets (CLC, HRL, CAMS, C3S) require authentication and
async downloads. For thesis structure, we define expected interfaces.
"""

copernicus_features = {
    "clc_industrial_area_km2": None,
    "mean_imperviousness_pct": None,
    "mean_solar_radiation": None
}

# =============================================================================
# 7. ENERGY FEATURE SUMMARY (NUMERIC OUTPUT)
# =============================================================================

energy_features = {
    "num_flat_roof_buildings": int(len(flat_roofs)),
    "num_power_objects": int(len(power_features)),
    "num_solar_generators": int(len(solar_generators)),
    "num_ev_charging_stations": int(len(charging_stations)),
    "num_pipelines": int(len(pipelines)),
    "num_viirs_fire_events": int(len(frp_gdf)) if frp_gdf is not None else 0
}

logging.info("Energy feature vector:")
for k, v in energy_features.items():
    logging.info(f"  {k}: {v}")

# =============================================================================
# 8. FOLIUM MAP (TILED, LAYERED OUTPUT)
# =============================================================================

map_center = [(NORTH + SOUTH) / 2, (EAST + WEST) / 2]
m = folium.Map(location=map_center, zoom_start=14, tiles="CartoDB positron")

# ---- ROI
folium.GeoJson(
    roi_gdf,
    name="ROI",
    style_function=lambda _: {
        "color": "black",
        "weight": 2,
        "fillOpacity": 0
    }
).add_to(m)

# ---- Flat roofs
flat_fg = folium.FeatureGroup(name="Flat Roof Buildings")
for _, row in flat_roofs.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda _: {"color": "orange", "weight": 1}
    ).add_to(flat_fg)
flat_fg.add_to(m)

# ---- Power infrastructure
power_fg = folium.FeatureGroup(name="Power Infrastructure")
for _, row in power_features.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda _: {"color": "blue", "weight": 1}
    ).add_to(power_fg)
power_fg.add_to(m)

# ---- EV charging stations
ev_fg = folium.FeatureGroup(name="EV Charging Stations")
for _, row in charging_stations.iterrows():
    if row.geometry.geom_type == "Point":
        folium.CircleMarker(
            [row.geometry.y, row.geometry.x],
            radius=5,
            color="green",
            fill=True,
            fill_opacity=0.8
        ).add_to(ev_fg)
ev_fg.add_to(m)

# ---- VIIRS FRP
if frp_gdf is not None:
    frp_fg = folium.FeatureGroup(name="VIIRS Fire Radiative Power")
    for _, row in frp_gdf.iterrows():
        folium.CircleMarker(
            [row.geometry.y, row.geometry.x],
            radius=3,
            color="red",
            fill=True,
            fill_opacity=0.6,
            tooltip=f"FRP: {row.get('FRP', 'n/a')} MW"
        ).add_to(frp_fg)
    frp_fg.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

# =============================================================================
# 9. SAVE OUTPUT
# =============================================================================

OUTPUT_HTML = "energy_features_KIT_North.html"
m.save(OUTPUT_HTML)
logging.info(f"Map saved to {OUTPUT_HTML}")


INFO:root:Downloading OSM features...


TypeError: features_from_bbox() got an unexpected keyword argument 'north'